In [ ]:
from gorillatracker.classification.clustering import (
    batch_visualize_alg_metrics,
)
from gorillatracker.classification.metrics import formatted_names
import pandas as pd

metrics_df = pd.read_pickle("/workspaces/gorillatracker/sep26_clustering_results.pkl")
# visualize_alg_metrics(metrics_df, "SPAC", "ViT-Finetuned", "KMeans", formatted_names)
batch_visualize_alg_metrics(
    metrics_df,
    "SPAC+min3",  # "Bristol",
    ["ViT-Pretrained", "ViT-Finetuned", "EfN-Pretrained"],
    ["KMeans", "AgglomerativeClustering"],
    formatted_names,
)

In [ ]:
print(metrics_df.columns)
print(metrics_df.dataset.unique())

In [ ]:
import pandas as pd


def format_value(value):
    if pd.isna(value) or value == "nan":
        return "nan"
    elif isinstance(value, (int, float)):
        return f"{value:.3f}"
    else:
        return str(value)


def generate_first_graphic_data(df, dataset, model):
    df = df.copy()

    # First, filter the DataFrame
    spac_max3_vit_df = df[(df["dataset"] == dataset) & (df["model"] == model)]

    # List to store the summary of metrics
    metrics_summary = []

    # List of algorithms to check
    algorithms_to_check = spac_max3_vit_df["algorithm"].unique()

    # Iterate over each algorithm
    for algorithm in algorithms_to_check:
        # Filter DataFrame for the current algorithm
        algorithm_df = spac_max3_vit_df[spac_max3_vit_df["algorithm"] == algorithm]

        # Get the rows with max silhouette score, max dunn index, min davies-bouldin index, and max calinski-harabasz index
        max_silhouette_row = algorithm_df.loc[algorithm_df["silhouette_coefficient"].idxmax()]
        max_dunn_row = algorithm_df.loc[algorithm_df["dunn_index"].idxmax()]
        min_davies_bouldin_row = algorithm_df.loc[algorithm_df["davies_bouldin_index"].idxmin()]
        max_calinski_harabasz_row = algorithm_df.loc[algorithm_df["calinski_harabasz_index"].idxmax()]

        # Add rows to the metrics summary
        metrics_summary.extend(
            [
                [
                    "Silhouette Score (max)",
                    algorithm,
                    max_silhouette_row["adjusted_rand_score"],
                    max_silhouette_row["homogeneity_score"],
                    max_silhouette_row["completeness_score"],
                    max_silhouette_row["v_measure_score"],
                    max_silhouette_row["n_clusters"],
                ],
                [
                    "Dunn Index (max)",
                    algorithm,
                    max_dunn_row["adjusted_rand_score"],
                    max_dunn_row["homogeneity_score"],
                    max_dunn_row["completeness_score"],
                    max_dunn_row["v_measure_score"],
                    max_dunn_row["n_clusters"],
                ],
                [
                    "Davies-Bouldin Index (min)",
                    algorithm,
                    min_davies_bouldin_row["adjusted_rand_score"],
                    min_davies_bouldin_row["homogeneity_score"],
                    min_davies_bouldin_row["completeness_score"],
                    min_davies_bouldin_row["v_measure_score"],
                    min_davies_bouldin_row["n_clusters"],
                ],
                [
                    "Calinski-Harabasz Index (max)",
                    algorithm,
                    max_calinski_harabasz_row["adjusted_rand_score"],
                    max_calinski_harabasz_row["homogeneity_score"],
                    max_calinski_harabasz_row["completeness_score"],
                    max_calinski_harabasz_row["v_measure_score"],
                    max_calinski_harabasz_row["n_clusters"],
                ],
            ]
        )

    # Convert to DataFrame for easy display
    return pd.DataFrame(
        metrics_summary,
        columns=[
            "Approach",
            "Algorithm",
            "Rand Score",
            "Homogeneity",
            "Completeness",
            "V-Measure",
            "n_clusters",
        ],
    )


def generate_latex_table(df, dataset, model):
    latex_code = [
        r"\begin{table}[H]",
        r"    \centering",
        r"    \resizebox{\textwidth}{!}{%",
        r"    \begin{tabular}{llccccc}",
        r"    \toprule",
        r"    \multicolumn{2}{l}{Approaches} & \multicolumn{5}{c}{Metrics} \\",
        r"     \cmidrule(lr){3-7}",
        r"    & & ARI & Homogeneity & Completeness & V-Measure & Clusters Found \\",
        r"    \midrule",
    ]

    for approach in df["Approach"].unique():
        latex_code.append(f"    \multicolumn{{7}}{{l}}{{\\textbf{{{approach}}}}} \\\\")
        group = df[df["Approach"] == approach]
        for _, row in group.iterrows():
            formatted_values = [
                format_value(row["Rand Score"]),
                format_value(row["Homogeneity"]),
                format_value(row["Completeness"]),
                format_value(row["V-Measure"]),
                str(row["n_clusters"]),
            ]
            latex_code.append(f"    & {row['Algorithm']} & {' & '.join(formatted_values)} \\\\")
        if approach != df["Approach"].unique()[-1]:
            latex_code.append(r"    \midrule")

    latex_code.extend(
        [
            r"    \bottomrule",
            r"    \end{tabular}%",
            r"    }",
            r"    \caption{Comparison of Clustering Approaches and Algorithms across Various Metrics. Dataset is "
            + dataset
            + " and Model is "
            + model
            + "}",
            r"    \label{tab:clustering-metrics}",
            r"\end{table}",
        ]
    )

    return "\n".join(latex_code)


dataset = "SPAC+min3"
model = "ViT-Finetuned"
metrics_summary_df = generate_first_graphic_data(metrics_df, dataset, model)
print(generate_latex_table(metrics_summary_df, dataset, model))

# Algorithms

In [ ]:
import pandas as pd
import numpy as np

def extract_data_best_algorithms_can_do(df):
    # Group by Dataset, Algorithm, and Model
    grouped = df.groupby(['dataset', 'algorithm', 'model'])
    
    report_data = []
    
    for (dataset, algorithm, model), group in grouped:
        # Find the row with the top ARI (adjusted_rand_score)
        top_ari_row = group.loc[group['adjusted_rand_score'].idxmax()]
        
        # Get the true number of clusters (assuming it's constant for each dataset-algorithm-model combination)
        true_clusters = group['n_true_clusters'].iloc[0]
        
        # Find the row with n_clusters closest to true_clusters
        closest_clusters_row = group.loc[(group['n_clusters'] - true_clusters).abs().idxmin()]
        
        report_data.append({
            'Dataset': dataset,
            'Algorithm': algorithm,
            'Model': model,
            'True Clusters': true_clusters,
            'ARI': top_ari_row['adjusted_rand_score'],
            'Clusters': top_ari_row['n_clusters'],
            'True ARI': closest_clusters_row['adjusted_rand_score']
        })
    
    return pd.DataFrame(report_data)

def format_value(value):
    if abs(value) < 1e-10:  # Consider values very close to zero as zero
        return "0"
    return f"{value:.3f}".rstrip('0').rstrip('.').replace('-0', '0')

def latex_table_best_algorithms_can_do(df):
    latex_code = [
        r"\begin{table}[H]",
        r"    \centering",
        r"    \resizebox{\textwidth}{!}{%",
        r"    \begin{tabular}{l|c|ccc|ccc|ccc}",
        r"    \toprule",
        r"    & True & \multicolumn{3}{c|}{KMeans} & \multicolumn{3}{c|}{AgglomerativeClustering} & \multicolumn{3}{c}{HDBSCAN} \\",
        r"    Embedding Space & Clusters & True ARI & ARI & Clusters & True ARI & ARI & Clusters & True ARI & ARI & Clusters \\",
        r"    \midrule",
    ]

    datasets = df['Dataset'].unique()
    for i, dataset in enumerate(datasets):
        group = df[df['Dataset'] == dataset]
        latex_code.append(f"    \\multicolumn{{11}}{{l}}{{\\textbf{{{dataset}}}}} \\\\")

        # Pivot the data
        pivot = group.pivot(index='Model', columns='Algorithm', 
                            values=['True Clusters', 'ARI', 'Clusters', 'True ARI'])
        
        for model in pivot.index:
            row = pivot.loc[model]
            true_clusters = row['True Clusters'].iloc[0]  # All algorithms have same True Clusters
            values = [
                model,
                str(int(true_clusters)),
            ]
            for alg in ['KMeans', 'AgglomerativeClustering', 'HDBSCAN']:
                values.extend([
                    format_value(row['True ARI'][alg]),
                    format_value(row['ARI'][alg]),
                    str(int(row['Clusters'][alg]))
                ])
            latex_code.append("    " + " & ".join(values) + r" \\")
        
        # Add a midrule between datasets, except after the last dataset
        if i < len(datasets) - 1:
            latex_code.append(r"    \midrule")

    latex_code.extend([
        r"    \bottomrule",
        r"    \end{tabular}%",
        r"    }",
        r"    \caption{Clustering Performance Metrics for Different Models and Algorithms}",
        r"    \label{tab:clustering-algorithms}",
        r"\end{table}",
    ])

    return "\n".join(latex_code)

# Example usage:
# Assuming metrics_df is your input DataFrame
report_df = extract_data_best_algorithms_can_do(metrics_df)
print(latex_table_best_algorithms_can_do(report_df))

In [ ]:
# NOTE(liamvdv): We're investiagting the performance of silouhette score -- maybe we didn't push the clustering far enough.
from gorillatracker.classification.clustering import (
    sweep_clustering_algorithms,
    configs,
    speed_level,
    visualize_alg_metrics,
    calculate_metrics,
    batch_visualize_alg_metrics,
    EXT_MERGED_DF,
    MERGED_DF,
    param_grid,
    speed,
)
from gorillatracker.classification.metrics import formatted_names
import pandas as pd

results2_df = sweep_clustering_algorithms(
    EXT_MERGED_DF,
    [("SPAC+min3", "ViT-Finetuned", "KMeans", param_grid({"n_clusters": range(3, 1000, speed(1, 5, 20))}))],
    cache_dir="./cache_cluster_sweep",
)
# visualize_alg_metrics(metrics_df, "SPAC", "ViT-Finetuned", "KMeans", formatted_names)
batch_visualize_alg_metrics(
    results2_df,
    "SPAC+min3",
    ["ViT-Pretrained", "ViT-Finetuned", "EfN-Pretrained"],
    ["KMeans", "AgglomerativeClustering"],
    formatted_names,
)